## Imports

In [2]:
!pip install GetOldTweets3
import GetOldTweets3 as got
import pandas as pd

import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer

import re
import json

tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+@#!?,;')

results = {}

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tamar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
with open('positive-words.txt') as f:
    p_txt = f.read()
    p_txt = re.sub('[,\.()":;!@#$%^&*\d]|\'s|\'', '', p_txt)
    p_list = p_txt.replace('\n',' ').replace('  ',' ').lower().split(' ')

In [5]:
with open('negative-words.txt', encoding = "ISO-8859-1") as f:
    n_txt = f.read()
    n_txt = re.sub('[,\.()":;!@#$%^&*\d]|\'s|\'', '', n_txt)
    n_list = n_txt.replace('\n',' ').replace('  ',' ').lower().split(' ')

## Function to Scrape Tweets

In [6]:
def text_query_to_csv(text_query, count, startDate, endDate ):
    
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(text_query).setMaxTweets(count).setSince(startDate)\
                                           .setUntil(endDate)
                                                    
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)

    text_tweets = [[tweet.date, tweet.text] for tweet in tweets]

    tweets_df = pd.DataFrame(text_tweets, columns = ['Datetime', 'Text'])

    tweets_df.to_csv('{}-tweets.csv'.format(text_query), sep=',')

## Preprocess the data

In [7]:
def dropNull_low_tok(df) -> pd.DataFrame:
    df['Text'].dropna(inplace=True)
    df['Text'] = df['Text'].str.lower()
    df['Text'] = df['Text'].astype(str)
    df['Text'] = df['Text'].apply(tokenizer.tokenize)
    return(df)

In [8]:
def remove_stops(row):
    my_list = row["Text"]
    meaningful_words = [w for w in my_list if not w in stop_words]
    return(meaningful_words)

In [9]:
def length_of_flat(wordlist):
    List_flat = []
    for i in range(len(wordlist)):
      for j in range (len(wordlist[i])):
        List_flat.append(wordlist[i][j])
    return(len(List_flat))

## Count positive and negative words

In [10]:
def tally(wordlist):
    word_count_dict = {}
    word_count_positive = {}
    word_count_negative= {}

    for tweet in range(len(wordlist)): 
        for word in wordlist[tweet]:
            # count all words frequency
            if word in word_count_dict.keys():
                word_count_dict[word] += 1
            else:
                word_count_dict[word] = 1
            # count if it is a positive word
            if word in p_list:
                if word in word_count_positive.keys():
                    word_count_positive[word] += 1
                else:
                    word_count_positive[word] = 1
            # else see if it is a negative word
            elif word in n_list:
                if word in word_count_negative.keys():
                    word_count_negative[word] += 1
                else:
                    word_count_negative[word] = 1
            else: 
                    pass
    pos = sum(word_count_positive.values())
    neg = sum(word_count_negative.values())
    return([pos, neg])

## Final output

In [11]:
def ratios(pos, neg, total):
    return((pos-neg)/total)

## Companies

In [65]:
companyName = "crown holdings"

#scrape tweets
text_query_to_csv(companyName,2500,"2015-01-01","2015-12-31")
x = pd.read_csv("C:\\Users\\Tamar\\Documents\\MMA\\Capstone Project\\"+companyName+"-tweets.csv")

x = dropNull_low_tok(x)
x['Text_Clean'] = x.apply(remove_stops, axis = 1)
x_list = x["Text_Clean"].values.tolist()
length = length_of_flat(x_list)

count = tally(x_list)
pos = count[0]
neg = count[1]
result = ratios(pos,neg,length)

results[companyName] = result

In [66]:
results

{'ceco environmental': 0.011554160125588697,
 'pinnacle west': 0.08140859407188386,
 'abbott laboratories': 0.009126844512510607,
 'aceto': 0.005400815217391304,
 'american airlines': 0.015550376342031864,
 'comdisco holding': 0.018137254901960786,
 'molson coors': 0.01812979515357152,
 'crown holdings': 0.0035293173979305367}

In [67]:
results = pd.DataFrame(results.items(), columns=['Company Name', 'Value'])

In [68]:
results.to_csv('twitterSentimentV3.csv')